### pulp+cbcを用いて、エッジを離散化して不在確率込みのTSPを解く

In [1]:
import random, math
import pulp
import itertools
import networkx as nx
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from itertools import combinations, permutations
import scop

%matplotlib inline

### create data

In [94]:
START_NODE = 0
NUM_NODE = 7
NUM_PERIOD = NUM_NODE + 2
END_NODE = NUM_NODE + 1
AREA_SIZE = 20

In [95]:
list_node_with_start_end = list(range(START_NODE, NUM_NODE + 2))
list_node_with_start = list(range(START_NODE, NUM_NODE + 1))
list_node_with_end = list(range(1, NUM_NODE + 2))
list_node = list(range(1, NUM_NODE + 1))
list_period = list(range(0, NUM_PERIOD))

In [96]:
node_coordinate_map = {}
for n in list_node:
    node_coordinate_map[n] = (random.randint(0, AREA_SIZE), random.randint(0, AREA_SIZE))

In [97]:
node_period_proba_map = {}
for n in list_node_with_start_end:
    for t in list_period:
        if n==START_NODE or n==END_NODE or t==list_period[0] or t==list_period[-1]:
            proba = 0
        else:
            proba = random.random()
        node_period_proba_map[n, t] = proba

In [98]:
node_node_dist_map = {}
for n1 in list_node_with_start:
    for n2 in list_node_with_end:
        if n1==START_NODE or n2==END_NODE:
            dist = 0
        else:
            x = node_coordinate_map[n1]
            y = node_coordinate_map[n2]
            dist = euclidean(x, y)
        node_node_dist_map[n1, n2] = dist

### Mip-Modelの作成

In [99]:
model = pulp.LpProblem(name='time-dependence-tsp', sense=pulp.LpMinimize)
sense_map = {v: k for k, v in pulp.LpConstraintSenses.items()}

#### 変数の作成

In [100]:
x_tij = {}
for t in list_period:
    for i in list_node_with_start:
        for j in list_node_with_end:
            if i==j:
                continue
            
            v = pulp.LpVariable(cat=pulp.LpBinary, name='x_%s_%s_%s'%(t, i, j))
            x_tij[t, i, j] = v
            model.addVariable(v)
            
u_i = {}
for i in list_node_with_start_end:
    v = pulp.LpVariable(cat=pulp.LpInteger, name='u_%s'%i)
    u_i[i] = v
    model.addVariable(v)

#### ノード間に関する制約
$x_i$

In [101]:
for i in list_node:
    expr = pulp.lpSum(x_tij[t, j, i] for t in list_period for j in list_node_with_start if (t, j, i) in x_tij)
    constr = pulp.LpConstraint(e=expr, sense=sense_map['='], rhs=1, name='node_in_constr_%s'%i)
    model.addConstraint(constr)
    
    expr = pulp.lpSum(x_tij[t, i, j] for t in list_period for j in list_node_with_end if (t, i, j) in x_tij)
    constr = pulp.LpConstraint(e=expr, sense=sense_map['='], rhs=1, name='node_out_constr_%s'%i)
    model.addConstraint(constr)

In [102]:
constr = pulp.LpConstraint(e=u_i[START_NODE], sense=sense_map['='], rhs=list_period[0], name='start_constr')
model.addConstraint(constr)

constr = pulp.LpConstraint(e=u_i[END_NODE], sense=sense_map['='], rhs=list_period[-1], name='end_constr')
model.addConstraint(constr)

#### 訪問する地点の時刻を表す制約
$y_t$

In [103]:
for i in list_node_with_end:
    expr = pulp.lpSum(t * x_tij[t, j, i] for t in list_period for j in list_node_with_start if (t, j, i) in x_tij) - u_i[i]
    constr = pulp.LpConstraint(e=expr, sense=sense_map['='], rhs=0, name='x_u_time_constr_%s'%i)
    model.addConstraint(constr)

#### mtz制約
$x, y$

In [104]:
BigM = len(list_period)
for i in list_node_with_start:
    for j in list_node_with_end:
        if i==j:
            continue

        expr = u_i[i] - u_i[j] + BigM * pulp.lpSum(x_tij[t, i, j] for t in list_period)
        constr = pulp.LpConstraint(e=expr, sense=sense_map['<='], rhs=BigM - 1, name='mtz_%s_%s'%(i, j))
        model.addConstraint(constr)

#### 始点と終点に関する制約
$x, y$

In [105]:
expr = pulp.lpSum(x_tij[t, START_NODE, i] for t in list_period for i in list_node)
constr = pulp.LpConstraint(e=expr, sense=sense_map['='], rhs=1, name='start_x_constr')
model.addConstraint(constr)

expr = pulp.lpSum(x_tij[t, i, END_NODE] for t in list_period for i in list_node)
constr = pulp.LpConstraint(e=expr, sense=sense_map['='], rhs=1, name='end_x_constr')
model.addConstraint(constr)

#### 目的関数
$x, y$

In [106]:
objective = pulp.lpSum(
                            node_period_proba_map[j, t] * node_node_dist_map[i, j] * x_tij[t, i, j]
                            for t in list_period for i in list_node for j in list_node
                            if (t, i , j) in x_tij
                      )
model.setObjective(obj=objective)

In [107]:
%%time

model.writeLP('./edge_discrete_tsp.lp')
solver = pulp.PULP_CBC_CMD(msg=1, maxSeconds=120, fracGap=0.1, threads=None)
pulp.LpStatus[model.solve(solver=solver)]

Wall time: 1min 59s


In [108]:
# xの確認
print('---------x---------')
for (t, i, j) in x_tij:
    if round(pulp.value(x_tij[t, i, j]))==1:
        print(i, j, t)

# uの確認
print('--------u----------')
for i in u_i:
    print(i, round(pulp.value(u_i[i])))
        
# 目的関数値
print('--------objective-------')
print(pulp.value(model.objective))

---------x---------
0 2 1
2 6 2
6 4 3
4 1 4
1 3 5
3 7 6
7 5 7
5 8 8
--------u----------
0 0
1 4
2 1
3 5
4 3
5 7
6 2
7 6
8 8
--------objective-------
11.609549823891891


### SCOP-Model

In [109]:
model = scop.Model('time-dependent-tsp')

#### 変数の作成

In [110]:
# 変数を作る
x_i = {}
for i in list_node_with_start_end:
    v = model.addVariable(name='x_%s'%i, domain=list_period)
    x_i[i] = v

#### 始点と終点の期は固定する

In [111]:
start_constr = scop.Linear(name='start_constr', direction='=', rhs=1, weight='inf')
start_constr.addTerms(1, x_i[START_NODE], list_period[0])
model.addConstraint(start_constr)

end_constr = scop.Linear(name='end_constr', direction='=', rhs=1, weight='inf')
end_constr.addTerms(1, x_i[END_NODE], list_period[-1])
model.addConstraint(end_constr)

#### それぞれのノードは別々の値をとる制約

In [112]:
all_diff_constr = scop.Alldiff(name='all_diff', varlist=x_i.values(), weight='inf')
model.addConstraint(all_diff_constr)

#### 目的関数の設定

In [113]:
owd_constr = scop.Quadratic(name='owd_constr', direction='<=', rhs=0, weight=1)
for i, j in list(itertools.permutations(list_node, 2)):
    for t in list_period[1:]:
        coeff = int(node_period_proba_map[j, t] * node_node_dist_map[i, j])
        owd_constr.addTerms(coeff, x_i[i], t-1, x_i[j], t)
        
model.addConstraint(owd_constr)

In [114]:
%%time

model.Params.OutputFlag = 1
model.optimize()

solving using parameters: 
 
  TimeLimit =600 second 

  RandomSeed= 1 

  OutputFlag= 1 


 ================ Now solving the problem ================ 

# reading data ... done: 0.01(s)
penalty = 3/34 (hard/soft), time = 0.01(s), iteration = 0
# improving the initial solution greedily
penalty = 1/11 (hard/soft), time = 0.01(s), iteration = 0
# start tabu search
penalty = 0/35 (hard/soft), time = 0.01(s), iteration = 2
penalty = 0/28 (hard/soft), time = 0.01(s), iteration = 12
penalty = 0/20 (hard/soft), time = 0.01(s), iteration = 18
penalty = 0/17 (hard/soft), time = 0.01(s), iteration = 57
penalty = 0/15 (hard/soft), time = 0.01(s), iteration = 261
penalty = 0/12 (hard/soft), time = 0.01(s), iteration = 1085
penalty = 0/10 (hard/soft), time = 0.01(s), iteration = 1116
penalty = 0/8 (hard/soft), time = 0.03(s), iteration = 3340


# penalty = 0/8 (hard/soft)
# cpu time = 0.03/2.49(s)
# iteration = 3340/319731

[best solution]
x_0: 0
x_1: 4
x_2: 1
x_3: 5
x_4: 3
x_5: 7
x_6: 2
x_7: 6
x_8:

In [115]:
node_visitperiod_map = {}
for i in x_i:
    node_visitperiod_map[i] = int(x_i[i].value)

list_visit_node = [i[0] for i in sorted(node_visitperiod_map.items(), key=lambda x: x[1])]

In [116]:
list_visit_node

[0, 2, 6, 4, 1, 3, 7, 5, 8]

In [ ]:
model.Params.OutputFlag